In [34]:
import logging
import sys
import zipfile
from functools import partial
from pathlib import Path

import pandas as pd
from pandas.testing import assert_frame_equal
from scipy.stats import circmean
from tabulate import tabulate

from wind_up.caching import with_parquet_cache
from wind_up.combine_results import calc_net_uplift
from wind_up.constants import OUTPUT_DIR, PROJECTROOT_DIR, TIMESTAMP_COL, DataColumns
from wind_up.interface import AssessmentInputs
from wind_up.main_analysis import run_wind_up_analysis
from wind_up.models import PlotConfig, WindUpConfig
from wind_up.reanalysis_data import ReanalysisDataset

sys.path.append(str(PROJECTROOT_DIR))
from examples.helpers import download_zenodo_data, setup_logger

CACHE_DIR = PROJECTROOT_DIR / "cache" / "smarteole_example_data"

ANALYSIS_TIMEBASE_S = 600
CACHE_SUBDIR = CACHE_DIR / f"timebase_{ANALYSIS_TIMEBASE_S}"
CACHE_SUBDIR.mkdir(exist_ok=True, parents=True)

ENSURE_DOWNLOAD = 1
CHECK_RESULTS = 1
ZIP_FILENAME = "SMARTEOLE-WFC-open-dataset.zip"
MINIMUM_DATA_COUNT_COVERAGE = 0.5  # 50% of the data must be present

In [35]:
CHECK_RESULTS

1

In [36]:
analysis_output_dir = OUTPUT_DIR / "smarteole_example"
analysis_output_dir.mkdir(exist_ok=True, parents=True)
setup_logger(analysis_output_dir / "analysis.log")
logger = logging.getLogger(__name__)

In [37]:
download_zenodo_data(record_id="7342466", output_dir=CACHE_DIR, filenames={ZIP_FILENAME})

File C:\Users\aclerc\Documents\GitHub\wind-up\cache\smarteole_example_data\SMARTEOLE-WFC-open-dataset.zip already exists. Skipping download.
File C:\Users\aclerc\Documents\GitHub\wind-up\cache\smarteole_example_data\SMARTEOLE-WFC-open-dataset.zip already exists. Skipping download.
File C:\Users\aclerc\Documents\GitHub\wind-up\cache\smarteole_example_data\SMARTEOLE-WFC-open-dataset.zip already exists. Skipping download.
File C:\Users\aclerc\Documents\GitHub\wind-up\cache\smarteole_example_data\SMARTEOLE-WFC-open-dataset.zip already exists. Skipping download.


[WindowsPath('C:/Users/aclerc/Documents/GitHub/wind-up/cache/smarteole_example_data/SMARTEOLE-WFC-open-dataset.zip')]

In [38]:
from smarteole_example import _unpack_scada
scada_df = _unpack_scada(ANALYSIS_TIMEBASE_S)
scada_df.head()

,TurbineName,ActivePowerMean,ActivePowerSD,WindSpeedMean,WindSpeedSD,YawAngleMean,YawAngleMin,YawAngleMax,PitchAngleMean,GenRpmMean,AmbientTemp,ShutdownDuration
TimeStamp_StartFormat,,,,,,,,,,,,
2020-02-17 16:30:00,SMV1,2017.8566,57.0837,13.6164,1.2411,247.884662,239.930,259.686,6.0648,1800.5394,11.6191,0
2020-02-17 16:40:00,SMV1,1946.2472,91.7614,13.0139,1.0178,259.686000,259.686,259.686,3.5153,1798.7652,11.5493,0
2020-02-17 16:50:00,SMV1,1946.5069,98.3934,12.5818,0.9482,259.686000,259.686,259.686,2.1367,1799.5171,11.4929,0
2020-02-17 17:00:00,SMV1,1828.9365,139.5910,11.6641,0.9244,252.924035,249.808,259.686,0.8526,1799.4533,11.4735,0
2020-02-17 17:10:00,SMV1,1751.7035,170.2128,11.3144,1.0116,249.808000,249.808,249.808,0.2404,1799.2136,11.3937,0


In [39]:
scada_df.head()

,TurbineName,ActivePowerMean,ActivePowerSD,WindSpeedMean,WindSpeedSD,YawAngleMean,YawAngleMin,YawAngleMax,PitchAngleMean,GenRpmMean,AmbientTemp,ShutdownDuration
TimeStamp_StartFormat,,,,,,,,,,,,
2020-02-17 16:30:00,SMV1,2017.8566,57.0837,13.6164,1.2411,247.884662,239.930,259.686,6.0648,1800.5394,11.6191,0
2020-02-17 16:40:00,SMV1,1946.2472,91.7614,13.0139,1.0178,259.686000,259.686,259.686,3.5153,1798.7652,11.5493,0
2020-02-17 16:50:00,SMV1,1946.5069,98.3934,12.5818,0.9482,259.686000,259.686,259.686,2.1367,1799.5171,11.4929,0
2020-02-17 17:00:00,SMV1,1828.9365,139.5910,11.6641,0.9244,252.924035,249.808,259.686,0.8526,1799.4533,11.4735,0
2020-02-17 17:10:00,SMV1,1751.7035,170.2128,11.3144,1.0116,249.808000,249.808,249.808,0.2404,1799.2136,11.3937,0


In [40]:
from smarteole_example import _unpack_metadata
metadata_df = _unpack_metadata()
metadata_df.head()

,Name,Latitude,Longitude,TimeZone,TimeSpanMinutes,TimeFormat
0,SMV1,49.84975,2.80194,UTC,10,Start
1,SMV2,49.84661,2.80150,UTC,10,Start
2,SMV3,49.84347,2.80162,UTC,10,Start
3,SMV4,49.84033,2.80122,UTC,10,Start
4,SMV5,49.83719,2.80080,UTC,10,Start


In [41]:
from smarteole_example import _unpack_toggle_data
toggle_df = _unpack_toggle_data(ANALYSIS_TIMEBASE_S)
toggle_df.head()

,toggle_on,toggle_off,yaw_offset_command
TimeStamp_StartFormat,,,
2020-02-17 16:30:00+00:00,False,True,0.0
2020-02-17 16:40:00+00:00,False,True,0.0
2020-02-17 16:50:00+00:00,False,True,0.0
2020-02-17 17:00:00+00:00,True,False,0.0
2020-02-17 17:10:00+00:00,True,False,0.0


In [42]:
ANALYSIS_TIMEBASE_S

600

In [43]:
toggle_df_no_tz = toggle_df.copy()
toggle_df_no_tz.index = toggle_df_no_tz.index.tz_localize(None)
scada_df = scada_df.merge(toggle_df_no_tz["yaw_offset_command"], left_index=True, right_index=True, how="left")
scada_df["yaw_offset_command"] = scada_df["yaw_offset_command"].where(scada_df["TurbineName"] == "SMV6", 0)
del toggle_df_no_tz
scada_df.head()

,TurbineName,ActivePowerMean,ActivePowerSD,WindSpeedMean,WindSpeedSD,YawAngleMean,YawAngleMin,YawAngleMax,PitchAngleMean,GenRpmMean,AmbientTemp,ShutdownDuration,yaw_offset_command
TimeStamp_StartFormat,,,,,,,,,,,,,
2020-02-17 16:30:00,SMV1,2017.8566,57.0837,13.6164,1.2411,247.884662,239.930,259.686,6.0648,1800.5394,11.6191,0,0.0
2020-02-17 16:40:00,SMV1,1946.2472,91.7614,13.0139,1.0178,259.686000,259.686,259.686,3.5153,1798.7652,11.5493,0,0.0
2020-02-17 16:50:00,SMV1,1946.5069,98.3934,12.5818,0.9482,259.686000,259.686,259.686,2.1367,1799.5171,11.4929,0,0.0
2020-02-17 17:00:00,SMV1,1828.9365,139.5910,11.6641,0.9244,252.924035,249.808,259.686,0.8526,1799.4533,11.4735,0,0.0
2020-02-17 17:10:00,SMV1,1751.7035,170.2128,11.3144,1.0116,249.808000,249.808,249.808,0.2404,1799.2136,11.3937,0,0.0


In [44]:
reanalysis_dataset = ReanalysisDataset(
        id="ERA5T_50.00N_2.75E_100m_1hr",
        data=pd.read_parquet(Path("smarteole_data")/ "ERA5T_50.00N_2.75E_100m_1hr_20200201_20200531.parquet"),
    )
reanalysis_dataset.data.head()

,10_m_hws_mean_mps,10_m_hwd_mean_deg-n_true,100_m_hws_mean_mps,100_m_hwd_mean_deg-n_true,2_m_temp_mean_deg-c,2_m_dew_point_temp_deg-c,2_m_rh_mean_%,0_m_pres_mean_pa,sea_level_pres_mean_pa,boundary_layer_height_m,total_cloud_cover_%,instantaneous_surface_sensible_heat_flux_wpm2,friction_velocity_mps,cloud_base_height_m,surface_solar_radiation_downwards_wpm2,era5t
datetime_start_utc,,,,,,,,,,,,,,,,
2020-02-01 00:00:00+00:00,6.29,203.1,10.33,206.6,11.6,9.8,88.9,99621.8,100815.6,834.2,100.0,30.138,0.441049,204.6,0.0,0
2020-02-01 01:00:00+00:00,6.57,204.3,10.80,207.4,11.6,10.2,91.2,99547.5,100739.8,813.0,100.0,33.538,0.435240,30.8,0.0,0
2020-02-01 02:00:00+00:00,6.54,209.5,10.69,212.1,11.6,10.7,93.9,99486.1,100678.0,808.8,100.0,32.996,0.424339,30.8,0.0,0
2020-02-01 03:00:00+00:00,6.29,216.2,10.25,218.9,11.6,11.1,96.4,99432.0,100623.2,823.8,100.0,27.442,0.411508,30.8,0.0,0
2020-02-01 04:00:00+00:00,6.06,220.8,9.81,223.6,11.6,11.3,97.9,99341.0,100530.8,788.9,100.0,22.671,0.396724,30.8,0.0,0


In [45]:
reanalysis_dataset

ReanalysisDataset(id='ERA5T_50.00N_2.75E_100m_1hr', data=                           10_m_hws_mean_mps  10_m_hwd_mean_deg-n_true  \
datetime_start_utc                                                       
2020-02-01 00:00:00+00:00               6.29                     203.1   
2020-02-01 01:00:00+00:00               6.57                     204.3   
2020-02-01 02:00:00+00:00               6.54                     209.5   
2020-02-01 03:00:00+00:00               6.29                     216.2   
2020-02-01 04:00:00+00:00               6.06                     220.8   
...                                      ...                       ...   
2020-05-31 19:00:00+00:00               4.30                      44.9   
2020-05-31 20:00:00+00:00               4.32                      43.7   
2020-05-31 21:00:00+00:00               4.26                      42.5   
2020-05-31 22:00:00+00:00               4.12                      43.0   
2020-05-31 23:00:00+00:00               4.17           

In [46]:
wtg_map = {
        f"SMV{i}": {
            "name": f"SMV{i}",
            "turbine_type": {
                "turbine_type": "Senvion-MM82-2050",
                "rotor_diameter_m": 82.0,
                "rated_power_kw": 2050.0,
                "cutout_ws_mps": 25,
                "normal_operation_pitch_range": (-10.0, 35.0),
                "normal_operation_genrpm_range": (250.0, 2000.0),
                "rpm_v_pw_margin_factor": 0.05,
                "pitch_to_stall": False,
            },
        }
        for i in range(1, 7 + 1)
    }
northing_corrections_utc = [
    ("SMV1", pd.Timestamp("2020-02-17 16:30:00+0000"), 5.750994540354649),
    ("SMV2", pd.Timestamp("2020-02-17 16:30:00+0000"), 5.690999999999994),
    ("SMV3", pd.Timestamp("2020-02-17 16:30:00+0000"), 5.558000000000042),
    ("SMV4", pd.Timestamp("2020-02-17 16:30:00+0000"), 5.936999999999996),
    ("SMV5", pd.Timestamp("2020-02-17 16:30:00+0000"), 6.797253350869262),
    ("SMV6", pd.Timestamp("2020-02-17 16:30:00+0000"), 5.030130916842758),
    ("SMV7", pd.Timestamp("2020-02-17 16:30:00+0000"), 4.605999999999972),
]

wd_filter_margin = 3 + 7 * ANALYSIS_TIMEBASE_S / 600
cfg = WindUpConfig(
    assessment_name="smarteole_example",
    timebase_s=ANALYSIS_TIMEBASE_S,
    require_ref_wake_free=True,
    detrend_min_hours=12,
    ref_wd_filter=[207 - wd_filter_margin, 236 + wd_filter_margin],  # steer is from 207-236
    filter_all_test_wtgs_together=True,
    use_lt_distribution=False,
    out_dir=analysis_output_dir,
    test_wtgs=[wtg_map["SMV6"], wtg_map["SMV5"]],
    ref_wtgs=[wtg_map["SMV7"]],
    ref_super_wtgs=[],
    non_wtg_ref_names=[],
    analysis_first_dt_utc_start=pd.Timestamp("2020-02-17 16:30:00+0000"),
    upgrade_first_dt_utc_start=pd.Timestamp("2020-02-17 16:30:00+0000"),
    analysis_last_dt_utc_start=pd.Timestamp("2020-05-25 00:00:00+0000") - pd.Timedelta(seconds=ANALYSIS_TIMEBASE_S),
    lt_first_dt_utc_start=pd.Timestamp("2020-02-17 16:30:00+0000"),
    lt_last_dt_utc_start=pd.Timestamp("2020-05-25 00:00:00+0000") - pd.Timedelta(seconds=ANALYSIS_TIMEBASE_S),
    detrend_first_dt_utc_start=pd.Timestamp("2020-02-17 16:30:00+0000"),
    detrend_last_dt_utc_start=pd.Timestamp("2020-05-25 00:00:00+0000") - pd.Timedelta(seconds=ANALYSIS_TIMEBASE_S),
    years_for_lt_distribution=0,
    years_for_detrend=0,
    ws_bin_width=1.0,
    asset={
        "name": "Sole du Moulin Vieux",
        "wtgs": list(wtg_map.values()),
        "masts_and_lidars": [],
    },
    northing_corrections_utc=northing_corrections_utc,
    toggle={
        "name": "wake steering",
        "toggle_file_per_turbine": False,
        "toggle_filename": "SMV_offset_active_toggle_df.parquet",
        "detrend_data_selection": "use_toggle_off_data",
        "pairing_filter_method": "any_within_timedelta",
        "pairing_filter_timedelta_seconds": 3600,
        "toggle_change_settling_filter_seconds": 120,
    },
)
plot_cfg = PlotConfig(show_plots=True, save_plots=False, plots_dir=cfg.out_dir / "plots")

loaded WindUpConfig assessment_name: smarteole_example
loaded WindUpConfig assessment_name: smarteole_example
loaded WindUpConfig assessment_name: smarteole_example
loaded WindUpConfig assessment_name: smarteole_example
toggle analysis period (UTC): 2020-02-17 16:30 to 2020-05-25 00:00
toggle analysis period (UTC): 2020-02-17 16:30 to 2020-05-25 00:00
toggle analysis period (UTC): 2020-02-17 16:30 to 2020-05-25 00:00
toggle analysis period (UTC): 2020-02-17 16:30 to 2020-05-25 00:00
long term period (UTC): 2020-02-17 16:30 to 2020-05-25 00:00
long term period (UTC): 2020-02-17 16:30 to 2020-05-25 00:00
long term period (UTC): 2020-02-17 16:30 to 2020-05-25 00:00
long term period (UTC): 2020-02-17 16:30 to 2020-05-25 00:00
detrend period (UTC): 2020-02-17 16:30 to 2020-05-25 00:00
detrend period (UTC): 2020-02-17 16:30 to 2020-05-25 00:00
detrend period (UTC): 2020-02-17 16:30 to 2020-05-25 00:00
detrend period (UTC): 2020-02-17 16:30 to 2020-05-25 00:00


In [47]:
assessment_inputs = AssessmentInputs.from_cfg(
        cfg=cfg,
        plot_cfg=plot_cfg,
        toggle_df=toggle_df,
        scada_df=scada_df,
        metadata_df=metadata_df,
        reanalysis_datasets=[reanalysis_dataset],
        cache_dir=CACHE_SUBDIR,
    )

loading cached pickle C:\Users\aclerc\Documents\GitHub\wind-up\cache\smarteole_example_data\timebase_600\preprocess.pickle
loading cached pickle C:\Users\aclerc\Documents\GitHub\wind-up\cache\smarteole_example_data\timebase_600\preprocess.pickle
loading cached pickle C:\Users\aclerc\Documents\GitHub\wind-up\cache\smarteole_example_data\timebase_600\preprocess.pickle
loading cached pickle C:\Users\aclerc\Documents\GitHub\wind-up\cache\smarteole_example_data\timebase_600\preprocess.pickle


In [48]:
results_per_test_ref_df = run_wind_up_analysis(assessment_inputs)

test turbines: ['SMV6', 'SMV5']
test turbines: ['SMV6', 'SMV5']
test turbines: ['SMV6', 'SMV5']
test turbines: ['SMV6', 'SMV5']
ref list: ['SMV7']
ref list: ['SMV7']
ref list: ['SMV7']
ref list: ['SMV7']
turbines to test: ['SMV6', 'SMV5']
turbines to test: ['SMV6', 'SMV5']
turbines to test: ['SMV6', 'SMV5']
turbines to test: ['SMV6', 'SMV5']
filter_all_test_wtgs_together SMV5 set 400 rows [2.9%] to NA
filter_all_test_wtgs_together SMV5 set 400 rows [2.9%] to NA
filter_all_test_wtgs_together SMV5 set 400 rows [2.9%] to NA
filter_all_test_wtgs_together SMV5 set 400 rows [2.9%] to NA
C:\Users\aclerc\Documents\GitHub\wind-up\wind_up\plots\windspeed_drift_plots.py:26: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
changed 764 [16.2%] rows from toggle_on True to False because toggle_change_settling_filter_seconds = 120
changed 764 [16.2%] rows from toggle_on True to False because toggle_change_settling_filter_seconds = 120
changed 764 [16.2%] rows from

KeyboardInterrupt: 